In [1]:
import pandas as pd
import torch
import re
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
df_jan = pd.read_excel("Employee_Report.xlsx", sheet_name="Jan")
df_feb = pd.read_excel("Employee_Report.xlsx", sheet_name="Feb")
df_mar = pd.read_excel("Employee_Report.xlsx", sheet_name="Mar")

In [3]:
df_jan["Month"] = "Jan"
df_feb["Month"] = "Feb"
df_mar["Month"] = "Mar"

In [4]:
df = pd.concat([df_jan, df_feb, df_mar], ignore_index=True)

print("Columns Available:", df.columns.tolist())

Columns Available: ['Employee_ID', 'Employee_Name', 'Department', 'Salary', 'Profit', 'Month']


In [5]:
model_name = "Qwen/Qwen2.5-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)

print("SLM Loaded Successfully")

`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/434 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk and cpu.


SLM Loaded Successfully


In [9]:
question = input("Question :")

prompt = f"""
You are a Python pandas expert.

STRICT RULES:
- Output ONLY executable pandas code.
- No markdown.
- No explanation.
- Use dataframe name: df
- Return exactly ONE python expression.
- If comparing months, use Month column.
- If aggregation needed, use groupby.

Columns:
{df.columns.tolist()}

Question:
{question}

Answer:
"""

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=200,
    temperature=0.2,
    do_sample=True
)

generated_tokens = outputs[0][inputs["input_ids"].shape[1]:]
response = tokenizer.decode(generated_tokens, skip_special_tokens=True)

print("Raw Model Output:\n", response)

Question : which month has the better profit?


Raw Model Output:
 (df.groupby('Month')['Profit'].sum().idxmax())


In [10]:
code = re.sub(r"```python", "", response)
code = re.sub(r"```", "", code)
code = code.strip()

print("\nExecutable Code:\n", code)


Executable Code:
 (df.groupby('Month')['Profit'].sum().idxmax())


In [11]:
unsafe_keywords = ["import", "__", "exec", "eval", "open", "os", "sys"]

if any(word in code for word in unsafe_keywords):
    print("Unsafe code blocked ❌")
else:
    try:
        result = eval(code)
        print("\nFinal Answer:")
        print(result)
    except Exception as e:
        print("Execution Error:", e)


Final Answer:
Feb
